# Implementation of recommendation bot in MiWork 
 - 2021-06-17, Hyoungjoon Lim @ jupyter notebook (python3, iOS)
 - Find task/job/reports registered repeatedly with a regular cycle (2021 1H) 

## 1. Environment setting
### 1-1. Package import

In [109]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from datetime import datetime, timedelta

### 1-2. Function definition

In [2]:
def cos_sim(text1, text2):
    return dot(text1, text2)/(norm(text1)*norm(text2))

In [3]:
def matrix_to_ind(matrix):
    res = np.array([])
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            tmp = [int(i), int(j), matrix[i][j]]
            res = np.append(res, tmp)
    res = pd.DataFrame(res.reshape(int(len(res)/len(tmp)),int(len(tmp))))
    res = res.sort_values(by=[2], axis=0, ascending=False)
    res = res.reset_index(drop=True)
    res_m = res[res[2] >= 0.4]
    ind = list(dict.fromkeys(res_m[0]))
    return ind

In [156]:
def tfidf_to_related_txt(data):
    
    id_data = list(set(data[data.columns[9]]))
    
    RESULT_data = np.array([])

    for i in range(len(id_data)):

        tmp_data = data[data[data.columns[9]]==id_data[i]]
        
        tmp_data = tmp_data.sort_values(by=tmp_data.columns[1])
        
        tfidf_mat = tfidf.fit_transform(tmp_data[tmp_data.columns[1]])

        cosine_sim = linear_kernel(tfidf_mat, tfidf_mat)

        np.fill_diagonal(cosine_sim,0)

        ind = matrix_to_ind(cosine_sim)

        tmp_data = tmp_data.reset_index(drop=True)

        result = np.array([])
        
        if len(ind)<=2: continue

        for j in range(len(ind)):

            tmp_res = [id_data[i], tmp_data.loc[ind[j]][tmp_data.columns[5]], tmp_data.loc[ind[j]][tmp_data.columns[3]], tmp_data.loc[ind[j]][tmp_data.columns[1]]]

            result = np.append(result, tmp_res)

        RESULT_data = np.append(RESULT_data, result)

#         print(i)

    RESULT_data = RESULT_data.reshape(int(len(RESULT_data)/len(tmp_res)),int(len(tmp_res)))

    RESULT_data = pd.DataFrame(RESULT_data)   

    RESULT_data = RESULT_data.drop_duplicates()
    
    RESULT_data = RESULT_data.sort_values(by=[0,2,3], ascending=[True,True,True])
            
    RESULT_data = RESULT_data.reset_index(drop=True)

    return RESULT_data

In [151]:
def recommendation(RESULT_data):
    
    RESULT_data['period'] = 0 # evaluation of time interval

    for i in range(1,len(RESULT_data)):

        try: RESULT_data['period'][i] = datetime.strptime(RESULT_data.loc[i][2], '%Y-%m-%d %H:%M:%S') - datetime.strptime(RESULT_data.loc[i-1][2], '%Y-%m-%d %H:%M:%S')
        
        except: RESULT_data['period'][i] = datetime.strptime(RESULT_data.loc[i][2], '%Y-%m-%d %H:%M') - datetime.strptime(RESULT_data.loc[i-1][2], '%Y-%m-%d %H:%M')            
            
    
    RESULT_data['var'] = float(0) # rate of period fluctuation

    for i in range((len(RESULT_data)-1)):

        if RESULT_data['period'][i] == 0 or RESULT_data['period'][i] == timedelta(0): continue
            
        elif RESULT_data['period'][i] <= timedelta(days=1): continue
            
        else:

            RESULT_data['var'][i] = float(abs(RESULT_data['period'][i+1]-RESULT_data['period'][i])/RESULT_data['period'][i])

    
    RESULT_data['label'] = 0 # recommedation candidate

    for i in range(1,len(RESULT_data)):

        if RESULT_data[0][i-1]==RESULT_data[0][i] and 0 < RESULT_data['var'][i] <= 0.25 :  

            RESULT_data['label'][i] = 1

    
    for i in range(1,(len(RESULT_data)-1)):

        if RESULT_data['label'][i-1] == 0 and RESULT_data['label'][i] == 1 :
            
            RESULT_data['label'][i-1] = 1
            
        elif RESULT_data[0][i] == RESULT_data[0][i-1] and RESULT_data['label'][i] == 0 and RESULT_data['label'][i-1] == 1:
            
            RESULT_data['label'][i] = 1

    RESULT_data_sel = RESULT_data[RESULT_data['label']==1]
    
    return RESULT_data_sel

## 2. Data preprocessing
### 2-1. Data upload

In [81]:
working_dir = '/Users/hyoungjoonlim/Desktop/usingPython/data/'

task = pd.read_csv(working_dir+'2106업무_패턴찾기.csv',encoding='CP949')
task = task.drop(['업무중요도'],axis=1)
job = pd.read_csv(working_dir+'2106세부업무_패턴찾기.csv',encoding='CP949')
report = pd.read_csv(working_dir+'2106간단보고_패턴찾기.csv',encoding='CP949')
report = report[['액티비티번호','액티비티내용','액티비티등록년월','액티비티등록일시','액티비티등록일자','액티비티등록자B레벨조직명',
                 '액티비티등록자C레벨조직명','액티비티등록자D레벨조직명','액티비티등록자사원명','액티비티등록자사원번호','액티비티등록자직위명','액티비티등록자직책명']]

### 2-2. Identify data structure

In [67]:
task.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3020 entries, 0 to 4845
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   업무번호         3020 non-null   object
 1   업무명          3020 non-null   object
 2   업무등록년월       3020 non-null   int64 
 3   업무등록일시       3020 non-null   object
 4   업무등록일자       3020 non-null   object
 5   업무등록자B레벨조직명  2926 non-null   object
 6   업무등록자C레벨조직명  1696 non-null   object
 7   업무등록자D레벨조직명  3010 non-null   object
 8   업무등록자사원명     3010 non-null   object
 9   업무등록자사원번호    3020 non-null   int64 
 10  업무등록자직위명     3009 non-null   object
 11  업무등록자직책명     467 non-null    object
dtypes: int64(2), object(10)
memory usage: 306.7+ KB


In [68]:
job.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13218 entries, 0 to 21163
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   세부업무번호         13218 non-null  int64 
 1   세부업무명          13218 non-null  object
 2   세부업무등록년월       13218 non-null  int64 
 3   세부업무등록일시       13218 non-null  object
 4   세부업무등록일자       13218 non-null  object
 5   세부업무등록자B레벨조직명  12082 non-null  object
 6   세부업무등록자C레벨조직명  9284 non-null   object
 7   세부업무등록자D레벨조직명  13181 non-null  object
 8   세부업무등록자사원명     13181 non-null  object
 9   세부업무등록자사원번호    13218 non-null  int64 
 10  세부업무등록자직위명     12952 non-null  object
 11  세부업무등록자직책명     1418 non-null   object
dtypes: int64(3), object(9)
memory usage: 1.3+ MB


In [69]:
report.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4561 entries, 0 to 5700
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   액티비티번호         4561 non-null   int64 
 1   액티비티내용         4561 non-null   object
 2   액티비티등록년월       4561 non-null   int64 
 3   액티비티등록일시       4561 non-null   object
 4   액티비티등록일자       4561 non-null   object
 5   액티비티등록자B레벨조직명  4266 non-null   object
 6   액티비티등록자C레벨조직명  3445 non-null   object
 7   액티비티등록자D레벨조직명  4561 non-null   object
 8   액티비티등록자사원명     4561 non-null   object
 9   액티비티등록자사원번호    4561 non-null   int64 
 10  액티비티등록자직위명     4477 non-null   object
 11  액티비티등록자직책명     539 non-null    object
dtypes: int64(3), object(9)
memory usage: 463.2+ KB


In [70]:
task['업무명'].isnull().sum(), job['세부업무명'].isnull().sum(), report['액티비티내용'].isnull().sum()

(0, 0, 0)

## 3. Evaluation of cosine similarity

In [82]:
stopwords = ['.', '(', ')', ',', "'", '%', '-', 'X', ').', '×','의','자','에','안','번',
                      '#','호','을','이','다','만','로','가','를',' ']

tfidf = TfidfVectorizer(stop_words=stopwords)

### 3-1. TF-IDF construction & Extraction of related texts

In [157]:
Result_task = tfidf_to_related_txt(task)

Result_job = tfidf_to_related_txt(job)

Result_report = tfidf_to_related_txt(report)

In [62]:
Result_task.to_csv('Result_task_ordered_v2.csv', index=False, encoding='cp949')
Result_job.to_csv('Result_job_ordered_v2.csv', index=False, encoding='cp949')
Result_report.to_csv('Result_report_ordered_v2.csv', index=False, encoding='cp949')

### 3-2. Evaluation of time interval & Extraction of recommendation candidate

In [158]:
Result_task_sel = recommendation(Result_task)
Result_job_sel = recommendation(Result_job)
Result_report_sel = recommendation(Result_report)

<ipython-input-151-3a5d207f0356>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  try: RESULT_data['period'][i] = datetime.strptime(RESULT_data.loc[i][2], '%Y-%m-%d %H:%M:%S') - datetime.strptime(RESULT_data.loc[i-1][2], '%Y-%m-%d %H:%M:%S')
<ipython-input-151-3a5d207f0356>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RESULT_data['var'][i] = float(abs(RESULT_data['period'][i+1]-RESULT_data['period'][i])/RESULT_data['period'][i])
<ipython-input-151-3a5d207f0356>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

In [160]:
Result_task_sel.to_csv('Result_task_sel.csv', index=False, encoding='cp949')
Result_job_sel.to_csv('Result_job_sel.csv', index=False, encoding='cp949')
Result_report_sel.to_csv('Result_report_sel.csv', index=False, encoding='cp949')

In [159]:
Result_task_sel.head(25)

,0,1,2,3,period,var,label
11,1005309,법인영업부문,2021-01-05 09:52:16,(업무)(매출)수도권 1월 영업매출 관리,"-120 days, 23:11:09",0.000000,1
12,1005309,법인영업부문,2021-02-03 11:08:06,(업무)(매출)수도권 2월 영업매출 관리,"29 days, 1:15:50",0.007762,1
13,1005309,법인영업부문,2021-03-04 17:48:41,(업무)(매출)수도권3월 영업매출 관리,"29 days, 6:40:35",0.082167,1
14,1005309,법인영업부문,2021-04-05 10:13:29,(업무)(매출)수도권 4월 영업매출 관리,"31 days, 16:24:48",0.013307,1
15,1005309,법인영업부문,2021-05-06 16:31:09,(업무)(매출)수도권 5월 영업매출 관리,"31 days, 6:17:40",0.112778,1
16,1005309,법인영업부문,2021-06-03 10:11:50,(업무)(매출)수도권 6월 영업매출 관리,"27 days, 17:40:41",4.883544,1
41,1006087,법인영업부문,2021-01-29 14:13:10,(신규) 디셈버앤컴퍼니/임원배상/0.1억/0.1억(100%)/전략/조환희,0:03:58,0.000000,1
42,1006087,법인영업부문,2021-03-24 08:53:45,(신규) 한서대학교 / 플러스교육기관 / 총 0.5억 / 0.5억(100%) / 전...,"53 days, 18:40:35",0.171564,1
43,1006087,법인영업부문,2021-05-26 09:00:20,(신규) 카파네트웍스/재산종합/1.5억/0.75억(50%)/6.4./전략/일반총괄,"63 days, 0:06:35",0.888806,1
44,1006087,법인영업부문,2021-06-02 09:08:37,(신규) 위시스왓/재산종합/0.2억/0.2억(100%)/최초가입/전략/조환희,"7 days, 0:08:17",0.998518,1
